In [9]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Constants
START_DATE = "2023-04-01"
END_DATE = "2024-04-01"
RISK_FREE_RATE = 0.07015  # Annual risk-free rate
TRADING_DAYS = 252  # Approximate trading days in a year

# Download data
def download_data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)
    return data['Adj Close']

# Calculate daily returns
def calculate_returns(data):
    return data.pct_change().dropna()

# Calculate alpha and beta
def calculate_alpha_beta(stock_returns, market_returns):
    cov_matrix = np.cov(stock_returns, market_returns)
    beta = cov_matrix[0, 1] / cov_matrix[1, 1]
    alpha = np.mean(stock_returns) - beta * np.mean(market_returns)
    return alpha, beta

# Calculate Sharpe ratio
def calculate_sharpe_ratio(returns, risk_free_rate):
    excess_returns = returns - (risk_free_rate / TRADING_DAYS)
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    annualized_sharpe = sharpe_ratio * np.sqrt(TRADING_DAYS)
    return annualized_sharpe

# Calculate max drawdown (O(n) solution)
def calculate_max_drawdown(returns):
    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()
    return max_drawdown

# Main script
def main():
    # Download data
    tickers = ["^NSEI", "TATAMOTORS.NS"]
    data = download_data(tickers, START_DATE, END_DATE)

    # Split data into NIFTY50 and TATA MOTORS
    nifty50 = data["^NSEI"]
    tatamotors = data["TATAMOTORS.NS"]

    # Calculate daily returns
    nifty50_returns = calculate_returns(nifty50)
    tatamotors_returns = calculate_returns(tatamotors)

    # Alpha and Beta
    alpha, beta = calculate_alpha_beta(tatamotors_returns, nifty50_returns)

    # Sharpe Ratios
    nifty50_sharpe = calculate_sharpe_ratio(nifty50_returns, RISK_FREE_RATE)
    tatamotors_sharpe = calculate_sharpe_ratio(tatamotors_returns, RISK_FREE_RATE)

    # Max Drawdowns
    nifty50_drawdown = calculate_max_drawdown(nifty50_returns)
    tatamotors_drawdown = calculate_max_drawdown(tatamotors_returns)

    # Print results
    print("Alpha (TATAMOTORS):", alpha)
    print("Beta (TATAMOTORS):", beta)
    print("Sharpe Ratio (NIFTY50):", nifty50_sharpe)
    print("Sharpe Ratio (TATAMOTORS):", tatamotors_sharpe)
    print("Max Drawdown (NIFTY50):", nifty50_drawdown)
    print("Max Drawdown (TATAMOTORS):", tatamotors_drawdown)

if __name__ == "__main__":
    main()


[*********************100%***********************]  2 of 2 completed

Alpha (TATAMOTORS): 0.0024941817962396504
Beta (TATAMOTORS): 1.0940499885320352
Sharpe Ratio (NIFTY50): 1.9684722740023046
Sharpe Ratio (TATAMOTORS): 3.5659803275239415
Max Drawdown (NIFTY50): -0.06611908149387102
Max Drawdown (TATAMOTORS): -0.09511209857308509
